# M/L Commando Course, Cambridge 2018

This notebook builds a simple autoencoder as a curio.

In [ ]:
from copy import copy
from math import exp
from random import random, shuffle, choice, randint

from keras import Input
from numpy import array, mean
from pandas import concat
from pandas import DataFrame
from keras.models import Sequential
from keras.layers import Dense,Reshape

In [ ]:
import scipy
from scipy import misc
import numpy
from sklearn.preprocessing import MinMaxScaler
import imageio
from scipy import ndimage
from pylab import plt

critter = misc.face()
plt.imshow(critter)
plt.show()

critter_luminosity = critter[:, :, 0]
critter_average = numpy.median(critter, axis=2)

sc = MinMaxScaler(feature_range=(0,1))

print(critter.shape)

plt.imshow(critter_average)
plt.show()
print("scale the critter")
critter = scipy.ndimage.zoom(critter_average, 0.2, order=3)
print(critter.shape)
# critter = numpy.array(critter, dtype='uint8')
critter = sc.fit_transform(critter)
plt.imshow(critter)
#plt.show()

print([critter[0:10]])



In [ ]:
from keras import Model
#Use the Keras functional API to build an autoencoder
img_shape = critter.shape
img_flat_len = img_shape[0]*img_shape[1]
compression_factor = 10
repr_shape = (img_shape[0]//compression_factor) *(img_shape[1]//compression_factor) #SQUISH

# "encoded" is the encoded representation of the input
input_img = Input(shape=(img_flat_len,))
encoded = Dense(repr_shape, input_shape=(img_flat_len,), activation='relu', name="encoded")(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(img_flat_len, activation='sigmoid')(encoded)
# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

autoencoder.summary()

In [ ]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy', metrics=["mae"])

In [ ]:
squashed = critter.reshape(1,-1)
print(squashed.shape)
print(squashed.dtype)

print([squashed[0:10]])

In [ ]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='loss', patience=0, verbose=0, mode='auto')
autoencoder.fit(squashed, squashed, verbose=1, callbacks=[es], epochs=10, batch_size=10)

In [ ]:
reconstituted = autoencoder.predict(squashed)
print(reconstituted[0:10])

reconstituted = reconstituted.reshape(img_shape)
plt.imshow(reconstituted)

So, we can recover the raccoon in near-to-original quality.  What is going on in the hidden layer?

In [ ]:
from keras import backend as K

# use a lambda expression to create a function generator
# this takes a network and a value n, and returns a function to access the nth layer of that network
nth_layer_output_generator = lambda my_network, n: K.function([my_network.layers[0].input],[my_network.layers[n].output])
get_1st_layer = nth_layer_output_generator(autoencoder, 1)
# to use the function, we must then pass an input into the network
# the function also returns a list, so we have to take the [0]th value from the list...
layer_output = get_1st_layer([squashed])[0]
plt.imshow(layer_output.reshape((img_shape[0]//compression_factor),(img_shape[1]//compression_factor)))
